In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
import shutil
import tempfile
from PIL import Image
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import numpy as np
import time
import gc
from torchvision.models import resnet18, ResNet18_Weights

# =====================================
# Configuración general
# =====================================
usar_transfer_learning = True
num_classes = 6
batch_size = 32
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("✅ CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("🖥️ GPU activa:", torch.cuda.get_device_name(0))
else:
    print("⚠️ GPU no detectada. Estás usando CPU (muy lento)")

# =====================================
# 1. Preparar dataset
# =====================================
ruta_tipoA = r"D:\Codigo\DATASET_PINOS\CUPRESSUS MACROCARPAA"
ruta_tipoB = r"D:\Codigo\DATASET_PINOS\CUPRESSUS SEMPERVIRENS"
directorio_temporal = tempfile.mkdtemp()

for carpeta_principal in [ruta_tipoA, ruta_tipoB]:
    nombre_tipo = os.path.basename(carpeta_principal).replace(' ', '_')
    for clase in os.listdir(carpeta_principal):
        carpeta_clase = os.path.join(carpeta_principal, clase)
        if os.path.isdir(carpeta_clase):
            nombre_clase = clase.replace(' ', '_')
            nuevo_nombre = f"{nombre_tipo}_{nombre_clase}"
            destino = os.path.join(directorio_temporal, nuevo_nombre)

            if not os.path.exists(destino):
                shutil.copytree(carpeta_clase, destino)
            else:
                for archivo in os.listdir(carpeta_clase):
                    ruta_origen = os.path.join(carpeta_clase, archivo)
                    ruta_destino = os.path.join(destino, archivo)
                    if not os.path.exists(ruta_destino):
                        shutil.copy2(ruta_origen, ruta_destino)

print("✅ Imágenes copiadas")

# =====================================
# 2. Crear DataLoaders
# =====================================
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(directorio_temporal, transform=transform)
class_names = dataset.classes

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# =====================================
# 3. Definir el modelo (ResNet18)
# =====================================
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)

print(f"Modelo enviado a {device}")

# =====================================
# 4. Función de pérdida y optimizador
# =====================================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# =====================================
# 5. Entrenamiento
# =====================================
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_batches = len(train_loader)

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        progress = (batch_idx + 1) / total_batches * 100
        print(f"Epoch {epoch+1}/{num_epochs} - Lote {batch_idx + 1}/{total_batches} - Progreso: {progress:.2f}%")

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {avg_train_loss:.4f} - Val Loss: {avg_val_loss:.4f}")

# =====================================
# 6. Guardar la matriz de confusión y resultados
# =====================================
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión - ResNet18")
plt.savefig("matriz_confusion_resnet18.png")
plt.close()

# Guardar los resultados en un archivo
evaluation_file = "resultados_resnet18.txt"
with open(evaluation_file, "w") as file:
    report = classification_report(y_true, y_pred, target_names=class_names)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")

    file.write("Matriz de Confusión:\n")
    file.write(str(conf_matrix) + "\n\n")
    file.write("Reporte de Clasificación:\n" + report + "\n")
    file.write(f"Accuracy: {accuracy:.4f}\n")
    file.write(f"Precision (macro): {precision:.4f}\n")
    file.write(f"Recall (macro): {recall:.4f}\n")
    file.write(f"F1 Score (macro): {f1:.4f}\n")

print(f"✅ Resultados guardados en {evaluation_file}")


✅ CUDA disponible: True
🖥️ GPU activa: NVIDIA GeForce RTX 3060 Laptop GPU
✅ Imágenes copiadas
Modelo enviado a cuda
Epoch 1/5 - Lote 1/241 - Progreso: 0.41%
Epoch 1/5 - Lote 2/241 - Progreso: 0.83%
Epoch 1/5 - Lote 3/241 - Progreso: 1.24%
Epoch 1/5 - Lote 4/241 - Progreso: 1.66%
Epoch 1/5 - Lote 5/241 - Progreso: 2.07%
Epoch 1/5 - Lote 6/241 - Progreso: 2.49%
Epoch 1/5 - Lote 7/241 - Progreso: 2.90%
Epoch 1/5 - Lote 8/241 - Progreso: 3.32%
Epoch 1/5 - Lote 9/241 - Progreso: 3.73%
Epoch 1/5 - Lote 10/241 - Progreso: 4.15%
Epoch 1/5 - Lote 11/241 - Progreso: 4.56%
Epoch 1/5 - Lote 12/241 - Progreso: 4.98%
Epoch 1/5 - Lote 13/241 - Progreso: 5.39%
Epoch 1/5 - Lote 14/241 - Progreso: 5.81%
Epoch 1/5 - Lote 15/241 - Progreso: 6.22%
Epoch 1/5 - Lote 16/241 - Progreso: 6.64%
Epoch 1/5 - Lote 17/241 - Progreso: 7.05%
Epoch 1/5 - Lote 18/241 - Progreso: 7.47%
Epoch 1/5 - Lote 19/241 - Progreso: 7.88%
Epoch 1/5 - Lote 20/241 - Progreso: 8.30%
Epoch 1/5 - Lote 21/241 - Progreso: 8.71%
Epoch 1/5 -

In [7]:
for name, param in model.named_parameters():
    print(f"Nombre: {name}, Tamaño: {param.size()}, Requiere gradiente: {param.requires_grad}")


Nombre: conv1.weight, Tamaño: torch.Size([64, 3, 7, 7]), Requiere gradiente: False
Nombre: bn1.weight, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: bn1.bias, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.0.conv1.weight, Tamaño: torch.Size([64, 64, 3, 3]), Requiere gradiente: False
Nombre: layer1.0.bn1.weight, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.0.bn1.bias, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.0.conv2.weight, Tamaño: torch.Size([64, 64, 3, 3]), Requiere gradiente: False
Nombre: layer1.0.bn2.weight, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.0.bn2.bias, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.1.conv1.weight, Tamaño: torch.Size([64, 64, 3, 3]), Requiere gradiente: False
Nombre: layer1.1.bn1.weight, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.1.bn1.bias, Tamaño: torch.Size([64]), Requiere gradiente: False
Nombre: layer1.1